In [1]:
# LOOPS THROUGH DOCUMENTS PREVIOUSLY ADDED TO COLLECTION.
# USES row_id IN CSV AS IDENTIFIER.

In [2]:
import csv
import firebase_admin
import google.cloud
import pandas as pd
from firebase_admin import credentials, firestore
from random import randrange
import time

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [4]:
file_path = # set path

In [5]:
cred = credentials.Certificate("../secrets/ServiceAccountKey.json")
app = firebase_admin.initialize_app(cred)
store = firestore.client()


In [6]:
collection_name = "triviaContent"

In [7]:
# read csv to pandas
data = pd.read_csv(file_path) 

In [8]:
data.head(10)

,row_id,question,answer,category,language
0,0,"""Now is the winter of our discontent"" is a line from which Shakespearian play?",Richard III,Art and literature,en
1,1,"""Our Town"" is a play by whom?",Thornton Wilder,Art and literature,en
2,2,"""The Diary of Anne Frank"" was first published in English under what title?",The diary of a young girl,Art and literature,en
3,3,"A band of painted or sculpted decoration, often at the top of a wall.",A frieze,Art and literature,en
4,4,"A composition made of cut and pasted pieces of materials, sometimes with images added by the artist.",Collage,Art and literature,en
5,5,"A decorative art movement that emerged in the late nineteenth century. Characterized by dense assymmetrical ornamentation in sinuous forms, it is often symbolic and of an erotic nature.",Art nouveau,Art and literature,en
6,6,"A design style prevalent during the 1920s and 1930s, characterized by a sleek use of straight lines and slender forms.",Art deco,Art and literature,en
7,7,"A European movement beginning in France in the thirteenth century. The artworks are characterized by a linear, graceful, elegant style more naturalistic than that which had existed previously in Europe.",Gothic,Art and literature,en
8,8,"A European movement of the late eighteenth to mid-nineteenth century. In reaction to neoclassicism, it focused on emotion over reason, and on spontaneous expression.",Romanticism,Art and literature,en
9,9,"A European style developed in France in the late eleventh century. Its sculpture is ornamental, stylized and complex.",Romanesque,Art and literature,en


In [9]:
# for each row
    # try
        # get doc with same row_id
        # add new fields
        # add counts doc in subcollection
        # add contributions doc in subcollection
    # except
        # add error to pandas df log column
    # else
        # add "DONE" to pandas df log column
    # if counter % 500 == 0:
        # print(counter)

In [10]:
%%time
for row in data.index:
# for row in range(0, 6, 1):
    try:
        docId = store.collection(collection_name).where("row_id", "==", str(data.iloc[row,0])).get()[0].id

        # Parent ID as first field to subcollection docs.
        data_to_set_subcollections = {
            "qid" : docId,
        }

        # Fields exclude, random1 and random2 to each triviaContent doc [0, 2147483647] [2147483647 is 32-bit integer max].
        data_to_set_parent = {
            "exclude" : False,
            "random1" : randrange(0, 2147483648), # Note max + 1
            "random2" : randrange(0, 2147483648), # Note max + 1
            "importBatch" : 1,
        }

        # Add new fields to each triviaContent doc. Set here and not in csv in order to get number and boolean type in firestore. 
        store.collection(collection_name).document(docId).set(data_to_set_parent, merge = True)

            # Add sub-collection docs.
        store.collection(collection_name).document(docId).collection("logsTriviaContent").document("counts").set(data_to_set_subcollections)
        store.collection(collection_name).document(docId).collection("logsTriviaContent").document("contributions").set(data_to_set_subcollections)

        data.loc[row, "run_log"] = "DONE"
        
            
    except Exception as e:
        data.loc[row, "run_log"] = "ERROR --- " + str(e)
        
        
    finally:
        # Print row number at every 500th row
        if row % 500 == 0:
            print(row)

        # Save csv at every 1000th row
        if row % 1000 == 0:
            data.to_csv ("../data/run_log.csv", index = False, header=True)
            print("-------------------- RUN LOG SAVED --------------------")

        
print("-------------------- PROCESSING DONE --------------------")
data.to_csv ("../data/run_log.csv", index = False, header=True)
print("-------------------- RUN LOG SAVED --------------------")

0
-------------------- RUN LOG SAVED --------------------
500
1000
-------------------- RUN LOG SAVED --------------------
1500
2000
-------------------- RUN LOG SAVED --------------------
2500
3000
-------------------- RUN LOG SAVED --------------------
3500
4000
-------------------- RUN LOG SAVED --------------------
4500
5000
-------------------- RUN LOG SAVED --------------------
5500
6000
-------------------- RUN LOG SAVED --------------------
6500
7000
-------------------- RUN LOG SAVED --------------------
7500
8000
-------------------- RUN LOG SAVED --------------------
8500
9000
-------------------- RUN LOG SAVED --------------------
9500
10000
-------------------- RUN LOG SAVED --------------------
10500
11000
-------------------- RUN LOG SAVED --------------------
11500
12000
-------------------- RUN LOG SAVED --------------------
12500
13000
-------------------- RUN LOG SAVED --------------------
13500
14000
-------------------- RUN LOG SAVED --------------------
14500
1500

In [11]:
data

,row_id,question,answer,category,language,run_log
0,0,"""Now is the winter of our discontent"" is a line from which Shakespearian play?",Richard III,Art and literature,en,DONE
1,1,"""Our Town"" is a play by whom?",Thornton Wilder,Art and literature,en,DONE
2,2,"""The Diary of Anne Frank"" was first published in English under what title?",The diary of a young girl,Art and literature,en,DONE
3,3,"A band of painted or sculpted decoration, often at the top of a wall.",A frieze,Art and literature,en,DONE
4,4,"A composition made of cut and pasted pieces of materials, sometimes with images added by the artist.",Collage,Art and literature,en,DONE
...,...,...,...,...,...,...
61898,64534,What was the original intended use for the RC car prior to being a toy?,War,Entertainment and games,en,DONE
61899,64535,What was the name of the sacred river in Samuel Taylor Coleridge's Xanadu?,Alph,Art and literature,en,DONE
61900,64536,Who played James Bond in Diamonds Are Forever?,Sean Connery,Entertainment and games,en,DONE
61901,64537,Who is known as Slim Shady?,Marshall Mathers,Entertainment and games,en,DONE


In [14]:
data["run_log"].unique()

array(['DONE'], dtype=object)